In [1]:
import requests
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import json

In [6]:
API_KEY = "RGAPI-a300d021-38e9-4729-bf6f-c88c4482c5db"
GAME_NAME = "X8Scorpio8X"
TAG_LINE = "NA1"  # do not include "#"
REGION = "americas"

def get_puuid_from_riot_id(game_name, tag_line, api_key, region):
    # URL-encode the game name and tag line
    game_name_encoded = quote(game_name)
    tag_line_encoded = quote(tag_line)
    
    url = f"https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name_encoded}/{tag_line_encoded}"
    headers = {"X-Riot-Token": api_key}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data["puuid"]
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
puuid = get_puuid_from_riot_id(GAME_NAME, TAG_LINE, API_KEY, REGION)
if puuid:
    print("PUUID:", puuid)


PUUID: vrfSU4Nh3GYoxHi2hLXg_yOczDqdE8WulZzWDLl2RZu6wtmH-e8i629HTEVbu1Uej1CpJZ3JIfXu3w


In [7]:
PUUID = puuid
REGION = "americas"  # match routing region

def get_match_ids(puuid, api_key, region, count=20):
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {
        "start": 0,
        "count": count  # number of matches to retrieve (max 100 per call)
    }
    headers = {"X-Riot-Token": api_key}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()  # list of match IDs
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage
match_ids = get_match_ids(PUUID, API_KEY, REGION)
if match_ids:
    print("Recent Match IDs:", match_ids)


Recent Match IDs: ['NA1_5304201516', 'NA1_5304195215', 'NA1_5304159402', 'NA1_5303121184', 'NA1_5303081497', 'NA1_5303045364', 'NA1_5302859149', 'NA1_5302593363', 'NA1_5302584020', 'NA1_5301958274', 'NA1_5301679760', 'NA1_5301539074', 'NA1_5300777438', 'NA1_5300767414', 'NA1_5299827201', 'NA1_5299689498', 'NA1_5299540428', 'NA1_5299303966', 'NA1_5299258676', 'NA1_5299227403']


In [26]:
def parse_match_data(match_data):
    if match_data['info']['queueId'] != 420:  # Only solo/duo ranked games
        return []

    teams = match_data['info']['teams']
    participants = match_data['info']['participants']

    # Split participants by teamId
    team1_id = teams[0]['teamId']
    team2_id = teams[1]['teamId']
    team1_players = [p for p in participants if p['teamId'] == team1_id]
    team2_players = [p for p in participants if p['teamId'] == team2_id]

    def team_stats(team, players, other_players):
        def get_ping_sum(key):
            return sum(p.get(key, 0) for p in players)

        # Map team positions to column names
        position_map = {
            'TOP': 'top',
            'JUNGLE': 'jung',
            'MIDDLE': 'mid',
            'BOTTOM': 'bot',
            'UTILITY': 'sup'
        }

        
        # Champion name and items per role
        role_data = {}
        for p in players:
            pos = p.get('teamPosition')
            if pos not in position_map:
                continue
            role = position_map[pos]
            champ = p.get('championName')
            items = [p.get(f'item{i}', 0) for i in range(7)]
            role_data[role] = champ
            role_data[f'{role}Items'] = items


        return {
            'matchId': match_data['metadata']['matchId'],
            'teamId': team['teamId'],
            'win': team['win'],
            'baronKills': team['objectives']['baron']['kills'],
            'dragonKills': team['objectives']['dragon']['kills'],
            'riftHeraldKills': team['objectives']['riftHerald']['kills'],
            'totalGold': sum(p['goldEarned'] for p in players),
            'goldSpent': sum(p['goldSpent'] for p in players),
            'totalKills': sum(p['kills'] for p in players),
            'totalDeaths': sum(p['deaths'] for p in players),
            'totalTurretsTaken': sum(p.get('challenges', {}).get('turretTakedowns', 0) for p in players),
            'totalInhibitorsTaken': sum(p['inhibitorKills'] for p in players),
            'averageChampionExp': sum(p['champExperience'] for p in players) / len(players),
            'wardsPlaced': sum(p['wardsPlaced'] for p in players),
            'goldDiff': sum(p['goldEarned'] for p in players) - sum(p['goldEarned'] for p in other_players),
            'allInPings': get_ping_sum('allInPings'),
            'assistMePings': get_ping_sum('assistMePings'),
            'commandPings': get_ping_sum('commandPings'),
            'enemyMissingPings': get_ping_sum('enemyMissingPings'),
            'enemyVisionPings': get_ping_sum('enemyVisionPings'),
            'holdPings': get_ping_sum('holdPings'),
            'getBackPings': get_ping_sum('getBackPings'),
            'totalPings': (
                get_ping_sum('allInPings') +
                get_ping_sum('assistMePings') +
                get_ping_sum('commandPings') +
                get_ping_sum('enemyMissingPings') +
                get_ping_sum('enemyVisionPings') +
                get_ping_sum('holdPings') +
                get_ping_sum('getBackPings')
            ),
            'gameDurationMinutes': match_data['info']['gameDuration'] / 60,
            **role_data,
            'magicDPM': sum(p['magicDamageDealtToChampions'] for p in players) / (match_data['info']['gameDuration'] / 60),
            'physicalDPM': sum(p['physicalDamageDealtToChampions'] for p in players) / (match_data['info']['gameDuration'] / 60),
            'trueDPM': sum(p['trueDamageDealtToChampions'] for p in players) / (match_data['info']['gameDuration'] / 60)
        }

    return [
        team_stats(teams[0], team1_players, team2_players),
        team_stats(teams[1], team2_players, team1_players)
    ]


def collect_match_stats(match_ids, api_key):
    all_rows = []
    for i, match_id in enumerate(match_ids):
        url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
        headers = {"X-Riot-Token": api_key}
        
        while True:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                match_data = response.json()
                rows = parse_match_data(match_data)
                all_rows.extend(rows)
                break
            elif response.status_code == 429:
                print("Rate limit exceeded. Sleeping for 60 seconds...")
                time.sleep(60)
            else:
                print(f"Failed to retrieve match {match_id}: {response.status_code}")
                break

        time.sleep(1.3)

    return pd.DataFrame(all_rows)



In [29]:
#TESTER SESSION

names_taglines = {"X8Scorpio8X":"NA1"}
puuids = []
region = "americas"

# getting puuids of silver players
for GAME_NAME in names_taglines:
    puuids.append(get_puuid_from_riot_id(GAME_NAME, names_taglines[GAME_NAME], API_KEY, region))

# getting match_ids from players' recent games
match_ids = []
for puuid in puuids:
    match_ids.append(get_match_ids(puuid, API_KEY, region, count=20))

match_ids = np.array(match_ids).flatten().tolist()



# collecting match stats
scorpioData = collect_match_stats(match_ids, API_KEY)

#creating Coefficient of variation for each team's damage
scorpioData['damageCV'] = scorpioData[['magicDPM', 'physicalDPM', 'trueDPM']].std(axis=1) / scorpioData[['magicDPM', 'physicalDPM', 'trueDPM']].mean(axis=1)

In [34]:
scorpioData.to_csv("X8Scorpio8X_Data")
scorpioData.head()

,matchId,teamId,win,baronKills,dragonKills,riftHeraldKills,totalGold,goldSpent,totalKills,totalDeaths,...,mid,midItems,bot,botItems,sup,supItems,magicDPM,physicalDPM,trueDPM,damageCV
0,NA1_5304201516,100,True,1,3,0,96478,85900,58,44,...,Yone,"[3032, 3031, 3072, 3153, 6673, 3006, 3340]",Kaisa,"[3087, 3157, 3006, 3124, 3115, 4645, 3340]",Lux,"[3871, 6655, 3020, 3165, 4645, 3108, 3364]",2055.844068,2057.084746,485.532203,0.591706
1,NA1_5304201516,200,False,2,4,1,95555,82468,44,58,...,Akali,"[3089, 0, 3157, 4646, 4645, 3020, 3340]",Draven,"[3031, 6676, 3072, 3172, 3036, 6673, 3340]",Blitzcrank,"[3170, 3869, 3109, 3110, 3075, 8020, 3364]",1246.942373,2340.264407,322.657627,0.774950
2,NA1_5304195215,100,False,0,0,0,3454,2500,0,0,...,Lucian,"[1055, 2003, 0, 0, 0, 0, 3340]",NaN,NaN,Neeko,"[2003, 3865, 0, 0, 0, 0, 3340]",62.916667,197.916667,0.000000,1.163070
3,NA1_5304195215,200,True,0,0,0,3659,2450,0,0,...,Mel,"[3070, 2003, 0, 0, 0, 0, 3340]",MissFortune,"[1055, 2003, 2010, 0, 0, 0, 3340]",Lux,"[3865, 2003, 0, 0, 0, 0, 3340]",230.000000,280.416667,14.583333,0.806820
4,NA1_5304159402,100,True,1,3,0,72866,64075,41,36,...,Veigar,"[6655, 3116, 3089, 4645, 3135, 3041, 3364]",Jinx,"[1055, 3006, 6672, 3085, 3031, 0, 3340]",Xerath,"[3871, 6655, 3020, 4628, 4645, 0, 3340]",2789.973451,816.371681,179.973451,1.078273


Collecting data from people who are in silver rank

In [36]:
names_taglines = {"Paro":"0820", "MicDonuld":"NA1", "prohanp":"NA1", "eGirl Trainer": "queer", "iSejuani":"NA1", "Fartgas12":"NA1", "Guymandudemanguy": "Ocean", "7hunder3":"Lucky", "WHITEsmoke96":"NA1", "grinder enjoyer":"nocap", 
                  "Atomic":"8541", "LucQie":"NA1", "Konlen":"NA1", "Hide Inside Lux":"NA1", "Valdivian":"NA1", "Gray Goo":"NA1"}
puuids = []
region = "americas"

# getting puuids of silver players
for GAME_NAME in names_taglines:
    puuids.append(get_puuid_from_riot_id(GAME_NAME, names_taglines[GAME_NAME], API_KEY, region))

# getting match_ids from players' recent games
match_ids = []
for puuid in puuids:
    match_ids.append(get_match_ids(puuid, API_KEY, region, count=15))

match_ids = np.array(match_ids).flatten().tolist()

In [37]:
# collecting match stats
silver_data = collect_match_stats(match_ids, API_KEY)

#creating Coefficient of variation for each team's damage
silver_data['damageCV'] = silver_data[['magicDPM', 'physicalDPM', 'trueDPM']].std(axis=1) / silver_data[['magicDPM', 'physicalDPM', 'trueDPM']].mean(axis=1)

Rate limit exceeded. Sleeping for 60 seconds...


In [39]:
silver_data.to_csv("silver_data")
silver_data

,matchId,teamId,win,baronKills,dragonKills,riftHeraldKills,totalGold,goldSpent,totalKills,totalDeaths,...,mid,midItems,bot,botItems,sup,supItems,magicDPM,physicalDPM,trueDPM,damageCV
0,NA1_5303840007,100,True,0,2,0,30094,25458,21,7,...,Vex,"[1056, 3145, 3020, 1052, 0, 6655, 3340]",Senna,"[1055, 3087, 0, 3009, 0, 0, 3340]",Morgana,"[3158, 2003, 3871, 2055, 6653, 0, 3364]",1104.065217,1192.434783,149.739130,0.709066
1,NA1_5303840007,200,False,0,0,0,22704,20775,7,21,...,Kassadin,"[1054, 3070, 3118, 1001, 3802, 0, 3340]",Lucian,"[3508, 1001, 2055, 1055, 1037, 0, 3340]",Braum,"[3869, 3190, 0, 0, 2422, 0, 3364]",857.543478,885.065217,123.391304,0.694577
2,NA1_5303811968,100,False,1,3,1,81632,75050,34,43,...,AurelionSol,"[3040, 3089, 3135, 6653, 3116, 3175, 3340]",Ashe,"[3031, 3032, 3085, 3036, 3172, 1038, 3340]",Bard,"[3877, 3742, 6653, 3171, 3091, 0, 3364]",2096.187050,1407.673861,269.160671,0.733659
3,NA1_5303811968,200,True,1,4,0,81304,69805,42,35,...,Zyra,"[3041, 6653, 3020, 3157, 4645, 1058, 3363]",Jinx,"[3032, 3085, 3031, 3156, 3072, 3006, 3364]",Nautilus,"[3047, 3876, 3109, 3190, 3075, 1057, 3364]",2173.741007,1771.894484,154.484412,0.782077
4,NA1_5303777401,100,False,0,1,0,60988,57873,31,40,...,Sylas,"[1056, 3152, 3020, 4645, 3100, 0, 3340]",Smolder,"[1056, 3508, 3009, 3031, 3094, 1036, 3363]",Ashe,"[3179, 3877, 6696, 3009, 6690, 0, 3364]",1201.308980,2205.844749,299.391172,0.771894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,NA1_5281683648,200,False,0,1,0,59279,56700,24,47,...,Ryze,"[3040, 6657, 3020, 3116, 3147, 2022, 3363]",Jinx,"[1055, 3006, 3085, 6672, 3031, 1036, 3340]",Shaco,"[1001, 6655, 3871, 6653, 2508, 3802, 3364]",1654.229804,1937.160060,118.494732,0.791354
278,NA1_5278373384,100,True,1,5,1,91198,80225,51,52,...,Irelia,"[3124, 3748, 3302, 6672, 3181, 3153, 3340]",Veigar,"[3089, 6655, 3157, 4646, 3020, 3135, 3363]",Morgana,"[3107, 3157, 3137, 4628, 3158, 4629, 3364]",2327.936378,2179.282219,321.117455,0.694772
279,NA1_5278373384,200,False,1,2,0,84447,76933,52,51,...,Garen,"[6631, 3742, 3046, 3170, 3033, 1038, 3340]",Jhin,"[6676, 3170, 3031, 3036, 3094, 0, 3340]",Seraphine,"[3870, 3116, 3040, 3171, 4629, 1058, 3364]",965.040783,2785.913540,670.326264,0.777516
280,NA1_5278349815,100,True,1,3,0,79847,73658,41,42,...,Irelia,"[2021, 3153, 3173, 3091, 3181, 6333, 3340]",Jhin,"[3026, 6676, 3009, 3031, 3094, 3036, 3340]",Pantheon,"[3877, 3047, 6692, 6610, 3155, 3133, 3364]",338.669604,2949.621145,279.726872,1.282001


In [12]:
silver_data["totalInhibitorsTaken"]

0      0
1      0
2      1
3      2
4      0
      ..
275    0
276    2
277    1
278    1
279    1
Name: totalInhibitorsTaken, Length: 280, dtype: int64